3) enpoint:
- def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
- Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [25]:
import pandas as pd

# Leo el archivo que voy a utilizar para este endpoint
games = pd.read_parquet('../Dataset/games_clean.parquet')
items = pd.read_parquet('../Dataset/items_clean.parquet')

In [26]:
games.head(5)

,id,title,developer,release_date,genres,tags,price
0,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",4.99
1,643980,Ironbound,Secret Level SRL,2018-01-04,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...",0.00
2,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...",0.00
3,767400,弹炸人2222,彼岸领域,2017-12-07,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",0.99
4,772540,Battle Royale Trainer,Trickjump Games Ltd,2018-01-04,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...",3.99


In [27]:
items.head(5)

,user_id,items_count,steam_id,playtime_forever,id,item_name
0,76561197970982479,277,76561197970982479,6,10,Counter-Strike
1,js41637,888,76561198035864385,0,10,Counter-Strike
2,evcentric,137,76561198007712555,923,1200,Red Orchestra: Ostfront 41-45
3,Riot-Punch,328,76561197963445855,0,10,Counter-Strike
4,doctr,541,76561198002099482,1131,300,Day of Defeat: Source


In [28]:
#Creamos dos dataframes donde contengan las columnas que vamos a utilizar 
games = games[['genres', 'id','release_date']]
items = items[['user_id','playtime_forever', 'id']]

In [29]:
games.head(5)

,genres,id,release_date
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,2018-01-04
1,"[Free to Play, Indie, RPG, Strategy]",643980,2018-01-04
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,2017-07-24
3,"[Action, Adventure, Casual]",767400,2017-12-07
4,"[Action, Adventure, Simulation]",772540,2018-01-04


In [30]:
items.head()

,user_id,playtime_forever,id
0,76561197970982479,6,10
1,js41637,0,10
2,evcentric,923,1200
3,Riot-Punch,0,10
4,doctr,1131,300


In [7]:
#Combertimos el archivo a parquet
games.to_parquet('../Dataset/user_genre_games.parquet', compression='snappy', index=False)
items.to_parquet('../Dataset/user_genre_items.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

In [31]:
#Generamos dos variables para que sea más facil de analizar
ids_df1 = set(games['id'])
ids_df2 = set(items['id'])

# Encontrar intersección de IDs
common_ids = ids_df1.intersection(ids_df2)

# Contar el número de IDs comunes
count_common_ids = len(common_ids)

print(f"Número de IDs que aparecen en ambas columnas: {count_common_ids}")

Número de IDs que aparecen en ambas columnas: 477


Probando el endpoint

In [33]:
def UserForGenre(genre: str):
    """
    Action
    Endpoint 3

    - def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado 
      y una lista de la acumulación de horas jugadas por año de lanzamiento.

    - Ejemplo de retorno: 
    {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}
    , {Año: 2011, Horas: 23}]}
    """
    #games = pd.read_parquet('Dataset/user_genre_games.parquet')
    #items = pd.read_parquet('Dataset/user_genre_items.parquet')
    global games, items

    # Filtrando los juegos por el género dado
    games_genre = games[games['genres'].str.contains(genre, case=False, na=False)]
    
    # Combinando los DataFrames games e items
    filter_games = games_genre.merge(items, on='id', how='inner')
    
    # Añadiendo la columna del año de lanzamiento
    filter_games['release_year'] = filter_games['release_date'].dt.year
    
    # Agrupando por user_id y release_year para sumar el tiempo de juego
    user_year = filter_games.groupby(['user_id', 'release_year']).agg({'playtime_forever': 'sum'}).reset_index()
    
    # Encontrando el usuario con más horas jugadas
    total_playtime_per_user = user_year.groupby('user_id')['playtime_forever'].sum()
    max_user = total_playtime_per_user.idxmax()
    
    # Filtrando las horas jugadas por año para el usuario con más horas
    user_playtime_by_year = user_year[user_year['user_id'] == max_user][['release_year', 'playtime_forever']]
    
    # Renombrando las columnas para cumplir con el formato requerido
    user_playtime_by_year.rename(columns={'release_year': 'Año', 'playtime_forever': 'Horas jugadas'}, inplace=True)
    
    # Imprimiendo los datos para depuración

    # Convirtiendo el resultado a un formato de lista de diccionarios
    year_playtime = user_playtime_by_year.to_dict('records')
    
    # Devolviendo el resultado en el formato especificado
    return {"Usuario con más horas jugadas para Género " + genre: max_user, "Horas jugadas": year_playtime}

prueba = UserForGenre('Action')
prueba


{'Usuario con más horas jugadas para Género Action': 'crustyoldkiller',
 'Horas jugadas': [{'Año': 2004, 'Horas jugadas': 388621}]}